In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import torch
import os
import cv2
import matplotlib.pyplot as plt
from model import *
from loss import *
from dataset import *
from train import *
from utils import *
from PIL import Image
from skimage import io, transform
import torchvision.transforms as transforms
import torch.optim as optim

In [3]:
os.chdir(r"C:\Users\Mike's PC\Desktop\extra_curricular_data_science\object_detection\yolo")

In [4]:
seed = 123
torch.manual_seed(seed)

In [5]:
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0
EPOCHS = 100
NUM_WORKERS = 2
NUM_CLASSES = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "custom_overfit.pth.tar"
IMG_DIR = "custom_data\images"
LABEL_DIR = "custom_data\labels"

In [6]:
model = Yolov1(split_size=7, num_boxes=2, num_classes=NUM_CLASSES).to(DEVICE)

In [8]:
model

Yolov1(
  (darknet): Sequential(
    (0): CNNBlock(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (2): CNNBlock(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): CNNBlock(
      (conv): Conv2d(192, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
  

In [10]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [11]:
loss_fn = YoloLoss()

In [12]:
if LOAD_MODEL:
    load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

In [28]:
transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])

In [29]:
train_dataset = VOCDataset(
    "custom_data.txt",
    transform=transform,
    img_dir=IMG_DIR,
    label_dir=LABEL_DIR,
    C=NUM_CLASSES
)

In [30]:
image, label_matrix = train_dataset[0]

In [31]:
label_matrix.shape

torch.Size([7, 7, 12])

In [ ]:








test_dataset = VOCDataset(
    "custom_data.txt", transform=transform, img_dir=IMG_DIR, label_dir=LABEL_DIR,
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

for epoch in range(EPOCHS):
    # for x, y in train_loader:
    #    x = x.to(DEVICE)
    #    for idx in range(8):
    #        bboxes = cellboxes_to_boxes(model(x))
    #        bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
    #        plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

    #    import sys
    #    sys.exit()

    pred_boxes, target_boxes = get_bboxes(
        train_loader, model, iou_threshold=0.5, threshold=0.4, num_classes=NUM_CLASSES
    )

    mean_avg_prec = mean_average_precision(
        pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
    )
    print(f"Train mAP: {mean_avg_prec}")

    if mean_avg_prec > 0.7:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
        print(f"Saved checkpoint to {LOAD_MODEL_FILE}")
        exit()

    train_fn(train_loader, model, optimizer, loss_fn)